In [1]:
# test python inner func
x = -1

def innerfunc():
    print(x)

innerfunc()
x = 1
innerfunc()

-1
1


In [4]:
# test cost_kwargs
cost_kwargs = {}
cost_kwargs['c'] = 1
b = cost_kwargs.pop("a", None)
print(b)

None


In [2]:
# test parameters
def testparsfunc(par1, par2 = par1):
    print(par1)
    print(par2)

testparsfunc(1,2)

NameError: name 'par1' is not defined

In [5]:
import tensorflow as tf
x = tf.placeholder(shape = (4,2), dtype=tf.float32)
y = tf.nn.softmax(x)
z = tf.argmax(y, axis=1)
a = tf.cast(z, tf.float32)
b = tf.reduce_mean(a)

xipt = [[0,1],[0.1,0.5],[0.2,0.1],[0.5,0.5]]
with tf.Session() as sess:
    feed_dict = {
        x: xipt
    }
    x, y, z, a, b = sess.run((x,y,z,a,b), feed_dict = feed_dict)
    print(x)
    print(y)
    print(z)

[[ 0.   1. ]
 [ 0.1  0.5]
 [ 0.2  0.1]
 [ 0.5  0.5]]
[[ 0.26894143  0.7310586 ]
 [ 0.40131232  0.59868765]
 [ 0.52497917  0.4750208 ]
 [ 0.5         0.5       ]]
[1 1 0 0]


In [20]:
# test the weight
import tensorflow as tf
labels = tf.placeholder(shape = (None,2), dtype=tf.float32)
classes_distrib = tf.reduce_sum(labels, axis=0)
classes_distrib_inv = 1 / classes_distrib
classes_weights = classes_distrib_inv / tf.reduce_sum(class_distribution_inv)
weight_map = tf.reduce_sum(labels * classes_weights, axis=1)
#zero_weight = tf.reduce_sum(labels) / (tf.cast(tf.shape(labels)[0], dtype=tf.float32) * 2)
#one_weight = 1 - zero_weight

labels_ipt = [[0,1],[0,1],[0,1],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0]]
with tf.Session() as sess:
    feed_dict = {
        labels: labels_ipt
    }
    labels,classes_distrib,classes_weights,weight_map = sess.run((labels,classes_distrib,classes_weights,weight_map), feed_dict = feed_dict)
    print(labels)
    print(classes_distrib)
    print(classes_weights)
    print(weight_map)
    #print(test_reduce)

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
[ 7.  3.]
[ 0.29999998  0.69999999]
[ 0.69999999  0.69999999  0.69999999  0.29999998  0.29999998  0.29999998
  0.29999998  0.29999998  0.29999998  0.29999998]


In [1]:
import os
from PIL import Image
import numpy as np
import sys
from tqdm import *

class VOT2016_Data_Provider():
    def __init__(self,pathofvot2016):
        pathofinput = pathofvot2016 + '/input'
        pathofgroundtruth = pathofvot2016 + '/groundtruth'
        self.datanamelist = os.listdir(pathofgroundtruth)
        self.datanamesize = len(self.datanamelist)
        assert(self.datanamesize > 0)
        self.datalength = []
        self.inputdata = []
        self.gtdata = []
        self.maxsteps = -1
        self.minsteps = 99999999
        for idata in range(self.datanamesize):
            input_pic_dir = pathofinput + '/' + self.datanamelist[idata]
            input_gt_dir = pathofgroundtruth + '/' + self.datanamelist[idata]
            piclist = os.listdir(input_pic_dir)
            for inamen in range(len(piclist) - 1, -1,-1):
                iname = piclist[inamen]
                if (os.path.splitext(iname)[1] != '.jpg' and os.path.splitext(iname)[1] != '.png'):
                    piclist.remove(iname)
            piclist = sorted(piclist)
            gtlist = os.listdir(input_gt_dir)
            for inamen in range(len(gtlist) - 1, -1,-1):
                iname = gtlist[inamen]
                if (os.path.splitext(iname)[1] != '.jpg' and os.path.splitext(iname)[1] != '.png'):
                    gtlist.remove(iname)
            gtlist = sorted(gtlist)
            assert(len(gtlist) == len(piclist))
            datalength = len(gtlist)
            if datalength > self.maxsteps: self.maxsteps = datalength
            if datalength < self.minsteps: self.minsteps = datalength
            self.datalength.append(datalength)
            for inamen in range(datalength):
                piclist[inamen] = input_pic_dir + '/' + piclist[inamen]
                gtlist[inamen] = input_gt_dir + '/' + gtlist[inamen]
            self.inputdata.append(piclist)
            self.gtdata.append(gtlist)
        print(self.datalength)
        print(self.datanamelist)
        print('DataOK, loaded %d groups data.' % (len(self.datalength)))
        print('Max steps:%d' % (self.maxsteps))
        print('Min steps:%d' % (self.minsteps))
        self.pointer = 0
        #self.bagdata, self.baglabel = self.get_data(8)
    
    def get_data(self, dataidx):
        assert (0 <= dataidx and dataidx < self.datanamesize)
        datname = self.datanamelist[dataidx]
        inputnamelist = self.inputdata[dataidx]
        gtnamelist = self.gtdata[dataidx]
        steps = self.datalength[dataidx]
        im1 = Image.open(inputnamelist[0])
        im1_np = np.array(im1)
        nx = len(im1_np)
        ny = len(im1_np[0])
        channals = len(im1_np[0][0])
        assert(channals == 3)
        np.zeros(4)
        inputdata = np.zeros((steps, nx, ny, channals), dtype=np.float32)
        gtdata = np.zeros((steps, nx, ny), dtype = np.bool)
        print('loading data ' ,datname, '...')
        for istep in tqdm(range(steps)):
            im_ipt = Image.open(inputnamelist[istep])
            inputdata[istep] = np.array(im_ipt)
            im_gt = Image.open(gtnamelist[istep])
            gtdata[istep] = np.array(im_gt)
        gtdata = gtdata.reshape((steps*nx*ny))
        gtdata = gtdata.astype(np.int32)
        gtdataonehot = np.zeros((steps*nx*ny, 2), dtype=np.float32)
        gtdataonehot[np.arange(steps*nx*ny), gtdata] = 1
        gtdataonehot = gtdataonehot.reshape((steps,nx,ny,2))
        return (inputdata, gtdataonehot)

    def get_data_one_batch(self, dataidx):
        inputdata, gtdataonehot = self.get_data(dataidx)
        inputdata = inputdata.reshape([1] + list(np.shape(inputdata)))
        gtdataonehot = gtdataonehot.reshape([1] + list(np.shape(gtdataonehot)))
        # inputdata.dim:(batch_size = 1, steps, nx, ny, channals)
        # gtdata.dim:(batch_size = 1, steps, nx, ny, nclass)
        return (inputdata, gtdataonehot)

    def __call__(self, batch_size = 1):
        return self.bagdata, self.baglabel

In [2]:
    channels = 3
    n_class = 2
    dptest = VOT2016_Data_Provider('/home/cjl/data/vot2016')
    iptdata, gtdata = dptest.get_data_one_batch(8)

 18%|█▊        | 36/196 [00:00<00:00, 355.78it/s]

[240, 1500, 267, 339, 129, 175, 350, 402, 196, 312, 292, 138, 707, 365, 76, 41, 392, 131, 713, 393, 377, 742, 366, 164, 201, 465, 151, 365, 118, 519, 105, 341, 140, 539, 682, 131, 201, 100, 191, 326, 61, 366, 351, 251, 708, 366, 844, 120, 567, 310, 63, 999, 156, 661, 725, 158, 558, 291, 293, 225]
['gymnastics2', 'girl', 'hand', 'birds1', 'soccer2', 'book', 'bolt1', 'handball2', 'bag', 'tunnel', 'fernando', 'soldier', 'iceskater2', 'shaking', 'bmx', 'ball2', 'soccer1', 'crossing', 'pedestrian2', 'car2', 'handball1', 'car1', 'godfather', 'motocross1', 'sphere', 'gymnastics4', 'butterfly', 'tiger', 'gymnastics3', 'fish3', 'ball1', 'wiper', 'pedestrian1', 'birds2', 'fish4', 'singer3', 'marching', 'matrix', 'traffic', 'dinosaur', 'motocross2', 'fish1', 'singer1', 'sheep', 'helicopter', 'singer2', 'graduate', 'glove', 'gymnastics1', 'fish2', 'leaves', 'nature', 'racing', 'iceskater1', 'basketball', 'rabbit', 'road', 'octopus', 'bolt2', 'blanket']
DataOK, loaded 60 groups data.
Max steps:1500

100%|██████████| 196/196 [00:00<00:00, 348.11it/s]


In [3]:
iptdata = iptdata[:,0:10,:,:,:]
gtdata = gtdata[:,0:10,:,:,:]

In [16]:
#gtdata.shape
tt = np.ones(10 * 360 * 480)
tt

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [17]:
import tensorflow as tf
x = tf.placeholder(shape = (None,2), dtype=tf.float32)
t = tf.placeholder(shape = (None), dtype=tf.float32)
y = tf.nn.softmax(x)
z = tf.argmax(y, axis=1)
a = tf.cast(z, tf.float32)
b = tf.reduce_mean(tf.cast(tf.equal(a, t), dtype=tf.float32))
with tf.Session() as sess:
    feed_dict = {
        x: gtdata.reshape((10 * 360 * 480,2)),
        t: tt
    }
    x, y, z, a, t,b = sess.run((x,y,z,a,t,b), feed_dict = feed_dict)
print(a)
print(t)
print(b)

[ 0.  0.  0. ...,  0.  0.  0.]
[ 1.  1.  1. ...,  1.  1.  1.]
0.0383524
